In [ ]:
from ipydatagrid import DataGrid, Filter, Sort, ConditionalRenderer

In [ ]:
from json import load

with open('./cars.json') as fobj:  
    data = load(fobj)

In [ ]:
from bqplot import ColorScale, OrdinalColorScale

acceleration_color_scale = ColorScale(min=0, max=20)
horsepower_renderer = ConditionalRenderer('value', '<', 150, 'red', 'white')

renderers = {
    'Acceleration': {'background_color': acceleration_color_scale},
    'Displacement': {'text_color': ColorScale(min=97, max=455)},
    'Horsepower': {'background_color': horsepower_renderer},
    'Name': {'background_color': ConditionalRenderer('value', 'contains', 'chevrolet', 'green', 'white')},
    'Origin': {'background_color': OrdinalColorScale(
        colors=['DodgerBlue', 'SeaGreen', 'Yellow'], domain=['USA', 'Japan', 'Europe'])},
    'Year': {'background_color': 'green'},
}

datagrid = DataGrid(data=data, base_row_size=32, base_column_size=100, renderers=renderers)
datagrid

In [ ]:
horsepower_renderer.operator = '>'
horsepower_renderer.output_if_true = 'green'

In [ ]:
datagrid.transform([Filter('Origin', '=', 'Europe'), Sort('Displacement')])

In [ ]:
acceleration_color_scale.min = 12.5
acceleration_color_scale.max = 20.5

In [ ]:
datagrid.revert()

In [ ]:
datagrid.transform([Filter('Origin', '=', 'USA'), Filter('Acceleration', '<', 13), Sort('Acceleration')])

# Perfomance test: A million cells with ipydatagrid

In [ ]:
from random import uniform

n_rows = 1000
n_columns = 1000

data = {'data': [], 'schema': {}}

data['data'] = [[uniform(0, 1) for c in range(n_columns)] for r in range(n_rows)]
data['schema']['fields'] = [{'name': str(c), type:'number'} for c in range(n_columns)]

In [ ]:
from bqplot import ColorScale
from ipydatagrid import DataGrid

color_scale = ColorScale(min=0, max=1, scheme='viridis')

huge_datagrid = DataGrid(data=data, default_background_color_renderer=color_scale)

In [ ]:
huge_datagrid

In [ ]:
from ipywidgets import FloatSlider, link

slider = FloatSlider(value=color_scale.max, min=0, max=5)
link((color_scale, 'max'), (slider, 'value'))

slider

In [ ]:
color_scale.scheme = 'magma'

In [ ]:
conditional_renderer = ConditionalRenderer('value', '<', 0.5, '#fc8403', 'white')

conditional_huge_datagrid = DataGrid(data=data, default_background_color_renderer=conditional_renderer)
conditional_huge_datagrid

In [ ]:
from ipywidgets import FloatSlider, Dropdown, ColorPicker, HBox, link

operator_dropdown = Dropdown(
    options=['<', '>', '='],
    value=conditional_renderer.operator
)
link((conditional_renderer, 'operator'), (operator_dropdown, 'value'))

reference_slider = FloatSlider(value=conditional_renderer.reference_value, min=0, max=1)
link((conditional_renderer, 'reference_value'), (reference_slider, 'value'))

output_colorpicker = ColorPicker(
    value=conditional_renderer.output_if_true
)
link((conditional_renderer, 'output_if_true'), (output_colorpicker, 'value'))

HBox((operator_dropdown, reference_slider, output_colorpicker))